In [2]:
import os
import re
import numpy as np
import pandas as pd
import rioxarray as rxr
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds

from glob import glob
from pathlib import Path

Define Paths

In [3]:
processed_data_dir = "../data/processed"
output_data_dir = "../data/outputs"

os.makedirs(output_data_dir, exist_ok=True)

Load/Align Rasters Helper Function

In [4]:
def load_and_align_rasters(file_list, reference_raster=None):
    """
    Loads a list of rasters and aligns them to a common resolution/extent.
    If reference_raster is provided, aligns all to that raster.
    Returns an xarray.Dataset with all rasters stacked as variables.
    """
    rasters = []
    names = []

    for file in file_list:
        arr = rxr.open_rasterio(file, masked=True).squeeze()
        names.append(os.path.splitext(os.path.basename(file))[0])

        # Reproject/align if reference is given
        if reference_raster is not None:
            arr = arr.rio.reproject_match(reference_raster)

        rasters.append(arr)

    # Stack into dataset
    ds = xr.merge([rasters[i].to_dataset(name=names[i]) for i in range(len(rasters))])
    return ds

Severity Classification Function

In [5]:
def classify_severity(dnbr):
    if dnbr < 0.1:
        return "Unburned"
    elif dnbr < 0.27:
        return "Low"
    elif dnbr < 0.44:
        return "Moderate"
    else:
        return "High"

Process All Fires

In [5]:
all_fire_dfs = []

# Detect fires based on available files
fires = sorted(set(os.path.basename(f).split("_")[0] for f in glob(os.path.join(processed_data_dir, "*.tif"))))

for fire_name in fires:
    print(f"Processing {fire_name} fire ...")

    fire_files = glob(os.path.join(processed_data_dir, f"{fire_name}_*.tif"))
    dnbr_path = os.path.join(processed_data_dir, f"{fire_name}_dNBR.tif")

    if not os.path.exists(dnbr_path):
        print(f"Skipping {fire_name} — no dNBR available.")
        continue

    try:
        reference = rxr.open_rasterio(dnbr_path, masked=True).squeeze()
        fire_ds = load_and_align_rasters(fire_files, reference_raster=reference)

        # Mask no-data areas based on dNBR
        fire_ds = fire_ds.where(~np.isnan(fire_ds[f"{fire_name}_dNBR"]), drop=True)

        # Convert to dataframe
        df = fire_ds.to_dataframe().reset_index()
        df = df.dropna()
        df["fire_name"] = fire_name
        df["severity"] = df[f"{fire_name}_dNBR"].apply(classify_severity)

        # Save individual file
        fire_output = os.path.join(output_data_dir, f"{fire_name}_dataset.parquet")
        df.to_parquet(fire_output, index=False)
        print(f"Saved {fire_name} dataset with {len(df)} pixels → {fire_output}")

        all_fire_dfs.append(df)

    except Exception as e:
        print(f"Error processing {fire_name}: {e}")



Processing Bootleg fire ...
Skipping Bootleg — no dNBR available.
Processing Caldor fire ...


C:\Users\John Waugh\AppData\Local\Temp\ipykernel_12704\1609434101.py:21: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  ds = xr.merge([rasters[i].to_dataset(name=names[i]) for i in range(len(rasters))])


KeyboardInterrupt: 

Combine Fires

In [6]:
fire_datasets = list(Path(output_data_dir).glob('*_dataset.parquet'))
output_file = Path("combined_dataset.parquet")

if output_file.exists():
    output_file.unlink()

writer = None
total_rows = 0

print(f"Found {len(fire_datasets)} fire datasets to combine.\n")

for i, file in enumerate(fire_datasets, start=1):
    fire_name = file.stem.replace('_dataset', '')
    print(f"[{i}/{len(fire_datasets)}] Processing {file.name}")

    dataset = ds.dataset(file, format="parquet")

    for batch in dataset.to_batches():
        df = batch.to_pandas()

        df.columns = [re.sub(f"^{fire_name}_", "", c) for c in df.columns]

        rename_map = {
            'x': 'longitude',
            'y': 'latitude',
            'veg_indices': 'NDVI'
        }
        df = df.rename(columns=rename_map)

        severity_cols = [c for c in df.columns if 'severity' in c.lower()]
        if severity_cols:
            df = df.rename(columns={severity_cols[0]: 'severity'})
        else:
            df['severity'] = pd.NA

        df['fire_name'] = fire_name

        expected_cols = ['latitude', 'longitude', 'dNBR', 'SPI', 'VCI', 'NDVI', 'severity', 'fire_name']
        for col in expected_cols:
            if col not in df.columns:
                df[col] = pd.NA

        df = df[expected_cols]

        for col in ['dNBR', 'SPI', 'VCI', 'NDVI']:
            df[col] = pd.to_numeric(df[col], errors='coerce', downcast='float')
        
        if 'severity' in df.columns:
            df['severity'] = df['severity'].astype('string')

        table = pa.Table.from_pandas(df, preserve_index=False)
        if writer is None:
            writer = pq.ParquetWriter(output_file, table.schema)
        writer.write_table(table)

        total_rows += len(df)

    print(f"   → Done ({total_rows:,} rows total so far)")

if writer:
    writer.close()

print(f"\nCombined dataset created successfully: {output_file}")
print(f"Total rows combined: {total_rows:,}")


Found 9 fire datasets to combine.

[1/9] Processing Caldor_dataset.parquet
   → Done (5,517,951 rows total so far)
[2/9] Processing Camp_dataset.parquet
   → Done (11,127,381 rows total so far)
[3/9] Processing Carr_dataset.parquet
   → Done (16,807,517 rows total so far)
[4/9] Processing Creek_dataset.parquet
   → Done (22,234,915 rows total so far)
[5/9] Processing Dixie_dataset.parquet
   → Done (27,860,550 rows total so far)
[6/9] Processing Glass_dataset.parquet
   → Done (31,693,542 rows total so far)
[7/9] Processing Thomas_dataset.parquet
   → Done (36,151,792 rows total so far)
[8/9] Processing Troublesome_dataset.parquet
   → Done (41,728,112 rows total so far)
[9/9] Processing Woolsey_dataset.parquet
   → Done (43,819,782 rows total so far)

Combined dataset created successfully: combined_dataset.parquet
Total rows combined: 43,819,782
